In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook as tqdm
from time import time
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from glove import Corpus, Glove
from multiprocessing import Pool
%matplotlib inline

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
DATA = pd.read_pickle('../Data_original/All_Data.pkl')
targets = pd.read_pickle("../Data_original/Targets.pkl")

In [4]:
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM

max_features = 100000
maxlen = 100 
batch_size = 32

model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(64))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              class_mode="binary")

model.fit(
    X_train, y_train, 
    batch_size=batch_size, 
    nb_epoch=1,
    show_accuracy=True
)

result = model.predict_proba(X)

Using TensorFlow backend.


NameError: name 'X_train' is not defined

In [ ]:
class LSTM(nn.Module):

    def __init__(self, input_dim, hidden_dim, batch_size, output_dim=1,
                    num_layers=2):
        super(LSTM, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers

        # Define the LSTM layer
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers)

        # Define the output layer
        self.linear = nn.Linear(self.hidden_dim, output_dim)

    def init_hidden(self):
        # This is what we'll initialise our hidden state as
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))

    def forward(self, input):
        # Forward pass through LSTM layer
        # shape of lstm_out: [input_size, batch_size, hidden_dim]
        # shape of self.hidden: (a, b), where a and b both 
        # have shape (num_layers, batch_size, hidden_dim).
        lstm_out, self.hidden = self.lstm(input.view(len(input), self.batch_size, -1))
        
        # Only take the output from the final timetep
        # Can pass on the entirety of lstm_out to the next layer if it is a seq2seq prediction
        y_pred = self.linear(lstm_out[-1].view(self.batch_size, -1))
        return y_pred.view(-1)

model = LSTM(lstm_input_size, h1, batch_size=num_train, output_dim=output_dim, num_layers=num_layers)

In [ ]:
loss_fn = torch.nn.MSELoss(size_average=False)

optimiser = torch.optim.Adam(model.parameters(), lr=learning_rate)

#####################
# Train model
#####################

hist = np.zeros(num_epochs)

for t in range(num_epochs):
    # Clear stored gradient
    model.zero_grad()
    
    # Initialise hidden state
    # Don't do this if you want your LSTM to be stateful
    model.hidden = model.init_hidden()
    
    # Forward pass
    y_pred = model(X_train)

    loss = loss_fn(y_pred, y_train)
    if t % 100 == 0:
        print("Epoch ", t, "MSE: ", loss.item())
    hist[t] = loss.item()

    # Zero out gradient, else they will accumulate between epochs
    optimiser.zero_grad()

    # Backward pass
    loss.backward()

    # Update parameters
    optimiser.step()

In [9]:
import pandas as pd
import numpy as np
import keras as K
from keras.layers import Input, Embedding, Dense, Dropout, merge, Flatten
from keras.regularizers import l2
from keras.models import Model
import keras.backend as K
from keras.optimizers import Adam
from keras.callbacks import Callback, TensorBoard

In [86]:
h1=10
h2=6
d1 = np.random.randint(low=0, high= h1, size=2000)
d2 = np.random.randint(low=0, high= h2, size=2000)
d3 = (d1 + d2) % 2

In [87]:
d = pd.DataFrame([d1, d2, d3]).T

In [88]:
np.random.seed = 42
msk = np.random.rand(len(d)) < 0.8
trn = d[msk]
val = d[~msk]

In [89]:
def rmse(y_true, y_pred):
    score = K.sqrt(K.mean(K.pow(y_true - y_pred, 2)))
    return score

In [90]:
from keras.layers.merge import concatenate

In [91]:
def model():
    u_ip = Input(shape=(1,), dtype='int64', name='user')
    m_ip = Input(shape=(1,), dtype='int64', name='movies')
    
    u = Embedding(h1, 1, input_length=1, embeddings_regularizer=l2(1e-3))(u_ip)
    m = Embedding(h2, 1, input_length=1, embeddings_regularizer=l2(1e-3))(m_ip)
    
    x = concatenate([u, m])
    x = Flatten()(x)

    x = Dropout(0.1)(x)
    x = Dense(1000, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(2)(x)
    
    model = Model([u_ip, m_ip], x)
    model.compile(Adam(5e-4), loss='mse', metrics=[rmse])
    return model

In [92]:
model = model()

In [93]:
model.fit([trn[0], trn[1]], trn[2], batch_size=8, epochs=20, 
          validation_data=([val[0], val[1]], val[2]))

Train on 1628 samples, validate on 372 samples
Epoch 1/20
1628/1628 [==============================] - 1s 799us/step - loss: 0.2785 - rmse: 0.5237 - val_loss: 0.2511 - val_rmse: 0.5011
Epoch 2/20
1628/1628 [==============================] - 1s 397us/step - loss: 0.2501 - rmse: 0.4999 - val_loss: 0.2561 - val_rmse: 0.5051
Epoch 3/20
1628/1628 [==============================] - 1s 386us/step - loss: 0.2500 - rmse: 0.4996 - val_loss: 0.2522 - val_rmse: 0.5021
Epoch 4/20
1628/1628 [==============================] - 1s 395us/step - loss: 0.2504 - rmse: 0.5000 - val_loss: 0.2515 - val_rmse: 0.5013
Epoch 5/20
1628/1628 [==============================] - 1s 387us/step - loss: 0.2490 - rmse: 0.4986 - val_loss: 0.2512 - val_rmse: 0.5010
Epoch 6/20
1628/1628 [==============================] - 1s 386us/step - loss: 0.2413 - rmse: 0.4908 - val_loss: 0.2038 - val_rmse: 0.4507
Epoch 7/20
1628/1628 [==============================] - 1s 386us/step - loss: 0.0948 - rmse: 0.2867 - val_loss: 0.0036 - val_

In [94]:
model.predict([val[0], val[1]])

array([[0.96820885],
       [0.9098037 ],
       [0.97065246],
       [0.06399336],
       [0.9685838 ],
       [0.05024562],
       [0.96271867],
       [0.01175106],
       [0.97065246],
       [0.01175106],
       [0.97278893],
       [0.97660303],
       [0.96290123],
       [0.05515822],
       [0.08329704],
       [0.07032101],
       [0.9114112 ],
       [0.08692215],
       [0.910103  ],
       [0.97065246],
       [0.06399336],
       [0.9026843 ],
       [0.02645183],
       [0.9020578 ],
       [0.90391177],
       [0.05901768],
       [0.911621  ],
       [0.965799  ],
       [0.9685838 ],
       [0.911621  ],
       [0.07410347],
       [0.04475382],
       [0.90391177],
       [0.96290123],
       [0.91145855],
       [0.96290123],
       [0.05981148],
       [0.05799556],
       [0.05214721],
       [0.91107005],
       [0.03777397],
       [0.97065246],
       [0.04400241],
       [0.97660303],
       [0.90346307],
       [0.90346307],
       [0.96608675],
       [0.972

In [ ]:
model.s